## DMM Result

In [1]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer
DMM = pickle.load(open('../../model_lda/DMM_result.pkl','rb'))

data = pickle.load(open('preprocessed_review.pickle', 'rb'))
data_join = [' '.join(doc) for doc in data]
cv = CountVectorizer()
X = cv.fit_transform(data_join).toarray()

DMM_lam = [DMM.components_[k,:] for k in range(4)]
def print_top_words(lam, feature_names, n_top_words):
    for topic_id, topic in enumerate(lam):
        print('\nTopic Nr.%d:' % int(topic_id + 1))
        print(''.join([feature_names[i] + ' ' + str(round(topic[i], 2))
                       + ' | ' for i in topic.argsort()[:-n_top_words - 1:-1]]))
print_top_words(DMM_lam, list(cv.get_feature_names()), 10)


Topic Nr.1:
hotel 3595.78 | stay 2803.74 | room 2764.67 | staff 1623.16 | great 1596.15 | service 1074.03 | waikiki 1015.44 | location 961.98 | beach 859.0 | place 831.88 | 

Topic Nr.2:
hotel 4903.6 | room 4674.14 | beach 3114.4 | stay 3002.51 | waikiki 2391.2 | view 2289.3 | great 2059.07 | nice 1425.27 | pool 1413.63 | location 1318.82 | 

Topic Nr.3:
hotel 7295.34 | great 5695.8 | room 4682.51 | stay 4162.87 | location 3840.33 | beach 3514.46 | staff 3322.45 | good 2555.8 | waikiki 2407.07 | nice 2009.29 | 

Topic Nr.4:
room 2199.08 | stay 2100.28 | hotel 1505.68 | staff 1337.29 | great 1107.37 | view 1024.0 | beach 826.53 | time 807.45 | waikiki 806.7 | location 740.27 | 


In [10]:
DMM.perplexity

[4071.136066896798,
 3537.59900784195,
 3533.560998627959,
 3532.17023724283,
 3531.920686136324,
 3531.6917306828714,
 3531.644966531001,
 3531.5823865185935,
 3531.4471804753357,
 3531.449035749878,
 3531.407825267577,
 3531.4084500608674,
 3531.408564245933,
 3531.408585340824]

## CLDA Result

In [2]:
CLDA = pickle.load(open('../../model_lda/CLDA_result.pkl','rb'))
CLDA_lam =  [CLDA.components_[k,:] for k in range(4)]
print_top_words(CLDA_lam, list(cv.get_feature_names()), 10)


Topic Nr.1:
get 3589.1 | price 2652.0 | go 2405.05 | time 2400.06 | day 2306.27 | resort 2213.1 | value 1917.0 | pay 1873.0 | fee 1791.0 | just 1709.18 | 

Topic Nr.2:
staff 8591.0 | pool 4981.0 | service 4875.0 | friendly 4705.0 | helpful 3495.0 | check 3270.0 | stay 3206.75 | desk 2724.0 | make 2513.1 | parking 2103.0 | 

Topic Nr.3:
great 10297.07 | beach 8314.1 | location 6861.1 | waikiki 6593.93 | restaurant 4208.0 | breakfast 3596.0 | walk 3320.1 | good 2879.16 | food 2667.0 | place 2579.82 | 

Topic Nr.4:
hotel 17017.05 | room 15320.0 | stay 9862.25 | view 5256.1 | nice 4207.09 | clean 3346.04 | close 3118.0 | front 3069.0 | ocean 2423.1 | bed 2215.1 | 


## CDMM Result

In [8]:
CLDA = pickle.load(open('../../model_lda/CDMM_result.pkl','rb'))
CLDA_lam =  [CLDA.components_[k,:] for k in range(4)]
print_top_words(CLDA_lam, list(cv.get_feature_names()), 20)


Topic Nr.1:
hotel 1321.47 | pay 1191.55 | price 1188.85 | fee 1167.34 | room 1163.75 | charge 1163.58 | value 1101.36 | rate 1097.24 | money 1092.0 | cost 1077.69 | expensive 1061.99 | cheap 1032.32 | cheaper 1026.83 | reasonably 1014.74 | pricey 1010.34 | fees 1001.0 | paid 1001.0 | priced 1000.98 | stay 817.48 | great 378.63 | 

Topic Nr.2:
hotel 8814.65 | room 7302.46 | stay 6273.04 | staff 4934.48 | great 4729.18 | beach 4581.76 | waikiki 3944.91 | service 3477.87 | view 3319.92 | pool 3156.33 | friendly 2916.09 | location 2867.34 | helpful 2370.82 | check 2325.41 | nice 2298.6 | clean 1978.27 | time 1939.65 | get 1895.03 | desk 1889.24 | good 1882.03 | 

Topic Nr.3:
hotel 5630.0 | great 4520.16 | room 3568.88 | stay 3117.61 | location 3029.76 | beach 2805.55 | restaurant 2492.46 | staff 2362.45 | good 2211.18 | breakfast 2017.49 | waikiki 1822.11 | food 1715.75 | nice 1599.1 | coffee 1473.56 | water 1456.43 | walk 1387.83 | clean 1315.62 | friendly 1297.91 | pool 1239.07 | drink 

In [9]:
model = pickle.load(open('../../model_lda/LDA_PAPER_result.pkl','rb'))
model_lam =  [model.components_[k,:] for k in range(4)]
print_top_words(model_lam, list(cv.get_feature_names()), 20)

[5327.934952413525,
 4516.49270583175,
 4504.4485755750975,
 4502.134875010409,
 4501.556036467643,
 4500.471026943269,
 4500.339380317486,
 4499.8820887317725,
 4499.746422329422,
 4499.545574818423,
 4499.480767985283,
 4499.443181234837,
 4499.4473468457845,
 4499.446479667193,
 4499.346261944583,
 4499.2145383856005,
 4499.2140385238345,
 4499.213999586309]

In [2]:
model = pickle.load(open('../../model_lda/LDA_PAPER_result.pkl','rb'))

In [4]:
model.delta_1

array([[1.        , 1.5       , 1.5       , ..., 1.5       , 1.5       ,
        1.5       ],
       [1.        , 1.        , 1.5       , ..., 1.5       , 1.        ,
        1.5       ],
       [1.5       , 1.        , 1.        , ..., 1.5       , 1.5       ,
        1.5       ],
       ...,
       [1.5       , 1.        , 1.5       , ..., 1.5       , 1.5       ,
        1.5       ],
       [1.        , 1.        , 1.        , ..., 1.49999999, 1.5       ,
        1.5       ],
       [1.5       , 1.        , 1.5       , ..., 1.5       , 1.5       ,
        1.5       ]])